# 14-9. 프로젝트 - Movielens 영화 추천 실습

1. 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
2. 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
3. 별점을 시청횟수로 해석해서 생각하겠습니다.
4. 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

### 내가좋아하는 영화
내가 좋아하는 영화는 다음과 같다고 설정 하였습니다. 
- Godfather, The
- Back to the Future
- Matrix
- Men in Black
- Jurassic Park

### 회고
영화 **대부**에 대한 선호도를 측정한 결과 는 0.62 이고, 좋아하는 영화 목록에 없는 **토이스토리**에 대한 선호도를 측정한 결는, 0.12 가 측정됨.<br>
**대부**와 유사한 추천영화는 장르 가 Action|Crime|Drama  인 장르의 영화를 추천받음.
좋아하는 영화의 장르에 대한 영화를 많이 추천 받았다. 

생각보다 pandas를 이용한 데이터 전처리부분만 신경을 쓰고 나머지 부분은 LMS와 거의 비슷하게 진행 하여 힘든 부분 은 적었다.
이상입니다. 

### 데이터 준비와 전처리

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, 
                      engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()


,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

print("뿡=3")

뿡=3


In [4]:
# ratings['user_id', 'movie_id', 'counts', 'timestamp']
ratings["counts"]

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
ratings.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head(10)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [7]:
# ratings : user_id, movie_id, ratings, timestamp
# movies  : movie_id, title, genre
    
    
    
    
    
    

### 데이터 둘러보기

In [8]:
# 유저 수
ratings['user_id'].nunique()

6039

In [9]:
# 영화 수 in ratings
ratings['movie_id'].nunique()

3628

In [10]:
# 영화 수 in movies
movies['movie_id'].nunique()

3883

In [11]:
# 인기 많은 영화
movie_data = pd.merge(ratings, movies)
movie_count = movie_data.groupby('title')['counts'].count()
movie_count.sort_values(ascending=False).head()

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Name: counts, dtype: int64

In [12]:
my_favorite = ['Godfather, The (1972)' , 'Back to the Future (1985)' ,'Matrix, The (1999)' ,'Men in Black (1997)' ,'Jurassic Park (1993)']
favorite_movie_id = movies[movies['title'].isin(my_favorite)]
my_movie = pd.DataFrame({'user_id': [6041]*5, 'movie_id': favorite_movie_id['movie_id'], 'counts':[5]*5})

if not ratings.isin({'user_id':[6041]})['user_id'].any():
    ratings = ratings.append(my_movie)
ratings.tail(10)

,user_id,movie_id,counts,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
476,6041,480,5,NaN
847,6041,858,5,NaN
1250,6041,1270,5,NaN
1539,6041,1580,5,NaN
2502,6041,2571,5,NaN


In [13]:
movie_data = pd.merge(ratings, movies, on='movie_id')
movie_data

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,9.783008e+08,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,9.782984e+08,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,9.782202e+08,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,9.781993e+08,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,9.781585e+08,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
836478,5851,3607,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western
836479,5854,3026,4,9.583469e+08,Slaughterhouse (1987),Horror
836480,5854,690,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance
836481,5938,2909,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary


### 데이터 전처리 완료

### CSR(Compressed Sparse Row) Matrix

In [14]:
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

ratings.user_id


0          1
1          1
2          1
3          1
4          1
        ... 
476     6041
847     6041
1250    6041
1539    6041
2502    6041
Name: user_id, Length: 836483, dtype: int64

In [15]:
ratings.movie_id

0       1193
1        661
2        914
3       3408
4       2355
        ... 
476      480
847      858
1250    1270
1539    1580
2502    2571
Name: movie_id, Length: 836483, dtype: int64

In [16]:
from scipy.sparse import csr_matrix

csr_data = csr_matrix((ratings['counts'], (ratings.user_id, ratings.movie_id)))
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 학습 시키기

In [17]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

csr_data_transpose = csr_data.T
csr_data_transpose
als_model.fit(csr_data_transpose)



  0%|          | 0/15 [00:00<?, ?it/s]

In [18]:
my_vector, godfather_vector = als_model.user_factors[6041], als_model.item_factors[858]
my_vector

array([ 0.42500046,  0.03993733, -0.49751818, -0.6029206 ,  0.2393402 ,
       -0.1032708 ,  0.18633533,  0.52579933,  0.15283169,  1.1453689 ,
       -0.20617606, -0.2899695 , -0.10439894, -0.45677426,  0.6882209 ,
       -0.61672026,  0.70725006,  0.17755912, -0.27103826,  0.91865027,
       -0.26168665, -0.03926421,  0.03921929,  0.49517128, -0.08113002,
        0.31370544, -0.22662449, -0.335473  ,  0.10385638,  0.19202276,
        0.90148395,  0.21389835, -0.70160276, -0.9602056 ,  0.26080143,
        0.29864454,  0.18628868,  0.05355492, -0.4120541 ,  0.14169858,
       -0.35514736, -0.89501375, -0.26797402,  0.43623954,  0.5851254 ,
       -0.24871582, -0.08481652,  0.58533156,  0.84998816, -0.23049031,
        0.21916232, -0.03539433, -0.47835463, -0.4658519 ,  1.6336108 ,
       -0.06834301,  0.7437817 ,  0.6429973 ,  0.3134563 ,  0.41205794,
       -0.49776757,  0.23334385,  0.02811623, -0.41241246, -0.11556379,
        0.5446314 , -0.6410658 ,  0.3142933 , -0.94708186, -0.18

In [19]:
godfather_vector

array([-4.37651295e-03,  4.05971296e-02, -3.65359634e-02,  9.36369688e-05,
        4.51399665e-03,  5.37008839e-03,  1.08872559e-02, -6.25095237e-03,
       -1.56994797e-02, -8.35034717e-03, -4.04855516e-03, -2.68046325e-03,
        1.65923666e-02, -1.86196566e-02,  1.97412912e-02,  3.77966906e-03,
        2.42991783e-02,  1.17302211e-02,  7.70029088e-04,  2.75972951e-02,
       -1.10347231e-03,  1.82772253e-03,  1.50738414e-02,  4.20955606e-02,
       -3.80222150e-03,  8.35134834e-03,  5.79732172e-02, -4.36014235e-02,
        1.06929848e-02,  4.69541289e-02,  1.63547099e-02, -4.77507347e-05,
       -2.50385571e-02, -3.03697344e-02,  1.53465988e-02,  6.73759878e-02,
        2.62162816e-02,  1.23616112e-02, -4.49764455e-04,  2.08668839e-02,
       -2.44543247e-04, -4.60585728e-02,  1.77271944e-02,  2.73248032e-02,
       -9.41422489e-03,  2.07556970e-02,  1.10650472e-02, -2.60688737e-02,
       -1.10817729e-02, -8.51792656e-03,  1.16900364e-02,  1.24803884e-02,
        1.44522395e-02,  

In [20]:
np.dot(my_vector, godfather_vector)

0.60457236

In [21]:
toystory_vector = als_model.item_factors[1]
np.dot(my_vector, toystory_vector)

0.12709087

### 취향이 비슷한 영화 찾기

In [22]:
favorite_movie = 'Godfather, The (1972)'
movie_id = movies[movies['title']=='Godfather, The (1972)']['movie_id']
similar_movie = als_model.similar_items(movie_id.values[0], N=15)
similar_movie

[(858, 1.0),
 (1221, 0.96712303),
 (2023, 0.5626301),
 (1953, 0.4630308),
 (3595, 0.35034597),
 (923, 0.34617645),
 (2695, 0.3449664),
 (1213, 0.34370944),
 (1787, 0.33223793),
 (624, 0.32572752),
 (1387, 0.3246959),
 (111, 0.32026905),
 (2887, 0.3055236),
 (912, 0.29692048),
 (1207, 0.2874818)]

In [23]:
movies[movies['movie_id'].isin([s[0] for s in similar_movie])]

,movie_id,title,genre
109,111,Taxi Driver (1976),Drama|Thriller
619,624,Condition Red (1995),Action|Drama|Thriller
847,858,"Godfather, The (1972)",Action|Crime|Drama
900,912,Casablanca (1942),Drama|Romance|War
911,923,Citizen Kane (1941),Drama
1189,1207,To Kill a Mockingbird (1962),Drama
1195,1213,GoodFellas (1990),Crime|Drama
1203,1221,"Godfather: Part II, The (1974)",Action|Crime|Drama
1366,1387,Jaws (1975),Action|Horror
1728,1787,Paralyzing Fear: The Story of Polio in America...,Documentary


### 추천하기

In [24]:
user = 6041
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(589, 0.62807435),
 (1221, 0.51933646),
 (2916, 0.40580946),
 (260, 0.38824803),
 (110, 0.3556946),
 (1573, 0.32851064),
 (1196, 0.32042235),
 (1210, 0.31530976),
 (1097, 0.29986787),
 (2028, 0.27774078),
 (1527, 0.27273428),
 (3175, 0.27245504),
 (1240, 0.26612544),
 (780, 0.26462275),
 (2628, 0.25999403),
 (1544, 0.25696608),
 (457, 0.25621223),
 (2023, 0.24728407),
 (648, 0.2275412),
 (2858, 0.21562073)]

In [25]:
movies[movies['movie_id'].isin([m[0] for m in movie_recommended])]

,movie_id,title,genre
108,110,Braveheart (1995),Action|Drama|War
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
453,457,"Fugitive, The (1993)",Action|Thriller
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
642,648,Mission: Impossible (1996),Action|Adventure|Mystery
770,780,Independence Day (ID4) (1996),Action|Sci-Fi|War
1081,1097,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
1178,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1203,1221,"Godfather: Part II, The (1974)",Action|Crime|Drama
